In [38]:
# Convert Markdown to Embeddings

from DailyVec import *

ocr_result_path = "../data/ocr/ocr_2024-06-03"
index_path = "../data/index/index_2024-06-03-fix"
convert_embeddings(ocr_result_path, index_path)

['../data/ocr/ocr_2024-06-03/2405-20467_1/2405-20467_1.md', '../data/ocr/ocr_2024-06-03/2405-20662_1/2405-20662_1.md', '../data/ocr/ocr_2024-06-03/2405-20715_1/2405-20715_1.md', '../data/ocr/ocr_2024-06-03/2405-20750_1/2405-20750_1.md', '../data/ocr/ocr_2024-06-03/2405-20630_1/2405-20630_1.md', '../data/ocr/ocr_2024-06-03/2405-20525_1/2405-20525_1.md', '../data/ocr/ocr_2024-06-03/2405-21009_1/2405-21009_1.md', '../data/ocr/ocr_2024-06-03/2405-21051_1/2405-21051_1.md', '../data/ocr/ocr_2024-06-03/2405-20559_1/2405-20559_1.md', '../data/ocr/ocr_2024-06-03/2405-20524_1/2405-20524_1.md', '../data/ocr/ocr_2024-06-03/2405-21040_1/2405-21040_1.md', '../data/ocr/ocr_2024-06-03/2405-20423_1/2405-20423_1.md', '../data/ocr/ocr_2024-06-03/2405-20456_1/2405-20456_1.md', '../data/ocr/ocr_2024-06-03/2405-21066_1/2405-21066_1.md', '../data/ocr/ocr_2024-06-03/2405-20502_1/2405-20502_1.md', '../data/ocr/ocr_2024-06-03/2405-20603_1/2405-20603_1.md', '../data/ocr/ocr_2024-06-03/2405-20980_1/2405-20980_1.m

/home/chivier/opt/miniconda3/envs/darxiv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [39]:
# Read vectors

vector_path = "../data/index/index_2024-06-03-fix/"

# Load all vectors

# find all npy files in the directory
import os
import numpy as np

def load_vectors(vector_path):
    vectors = {}
    for file in os.listdir(vector_path):
        if file.endswith(".npy"):
            local_vectors = np.load(vector_path + file)
            key_prefix = file.split(".")[0].replace("_vectors", "")
            chunk_id = 0
            for vector in local_vectors:
                chunk_id += 1
                vector_key = key_prefix + "_" + str(chunk_id)
                vectors[vector_key] = vector
    return vectors

vectors = load_vectors(vector_path)

In [40]:
print(len(vectors))

5431


In [41]:
question = "LLM model serving on distrubuted systems or serverless architecture. Related to VLLM or OLLAMA."

In [42]:
import os
from sentence_transformers import SentenceTransformer

import os
import csv
import torch
import json
import numpy as np


def get_embeddings(sentences):
    model = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)
    embeddings = model.encode(sentences)
    # release the memory of the model
    torch.cuda.empty_cache()
    return embeddings

question_embeddings = get_embeddings([question])


In [43]:
print(question_embeddings[0].shape)

(1024,)


In [44]:
def L2_distance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

def cos_sim(vector1, vector2):
    return -np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

def get_closest_vector(vectors, target_vector, top_k=3):
    # find the closest vector
    top_kv = []
    for key, vector in vectors.items():
        # print(key, L2_distance(vector, target_vector))
        top_kv.append((key, cos_sim(vector, target_vector)))
        top_kv.sort(key=lambda x: x[1])
        if len(top_kv) > top_k:
            top_kv.pop()
    return top_kv
    
question_result = get_closest_vector(vectors, question_embeddings[0], 10)

In [47]:
def read_source_from_key(source_path, key):
    chunk_id = key.split("_")[-1]
    # file_id = key remove "_{chunk_id}"
    file_id = key[:-(len(chunk_id) + 1)]
    # print(chunk_id, file_id)
    with open(source_path + file_id + "_chunks.csv", "r") as f:
        # read #chunk_id line from the file
        reader = csv.reader(f)
        for i, row in enumerate(reader):
            if i == int(chunk_id):
                return file_id, row[1]
        
    
for key, distance in question_result:
    # print(key, distance)
    source = read_source_from_key("../data/index/index_2024-06-03/", key)
    print(source[1])
    print("\n\n")

"We are not aware of any other serverless framework that supports a Wasm runtime and allows both the controller and the workers to be deployed on resource-constrained edge nodes.\n\nFocusing on serverless platforms supporting Wasm runtimes, Hall and Ramachandran were among the first to advocate WebAssembly as the enabling technology to avoid the overhead of containers, which would substantially weigh on the limited hardware resources of edge computing environments [10]. They presented a serverless platform that runs WebAssembly code within the V8 JavaScript engine for execution and sandboxing of functions. Differently from FunLess they use a NodeJS runtime which embeds V8 for the execution of WebAssembly code. As the authors note [10],\nthe nesting of these layers takes a conspicuous toll on the performance of the system.\n\nGadepalli et al. [11] use WebAssembly to run and sandbox serverless functions. They target only single-host deployments, requiring the deployment of the entire pla